In [1]:
import sys
sys.path.append('/home/gbortolai/Thesis/')

In [33]:
import torch
import numpy as np
import ot
# from multiprocessing import Pool
import torch.multiprocessing as mp
from gpu_worker_module import fun

In [56]:
#GPU
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

Using cpu device


def wasserstein_dist(a,b,metric, d):
    
    source = a[torch.nonzero(a, as_tuple = False)[:,0], torch.nonzero(a, as_tuple = False)[:,1]].view(torch.nonzero(a, as_tuple = False).size(0))
    target = b[torch.nonzero(b, as_tuple = False)[:,0], torch.nonzero(b, as_tuple = False)[:,1]].view(torch.nonzero(b, as_tuple = False).size(0))
    
    source = source.to(device)
    target = target.to(device)
    
    source = source/torch.sum(source, dtype = torch.double)
    target = target/torch.sum(target, dtype = torch.double)
    
    M = ot.dist(torch.nonzero(a, as_tuple = False), torch.nonzero(b, as_tuple = False), metric = metric).to(device)
    M = M/d
    
    T = ot.emd(source, target, M).to(device)
    W = torch.sum(T*M).to(device).to(device)
    
    W = torch.sqrt(W)
    
    return W

def fun(data_source, data_target, metric, d, ind1, ind2, device):
    
    torch.cuda.set_device(device)
    
    W = torch.zeros(data_source.size(0), data_target.size(0))
    
    if ind1 == ind2:
        for i in range(data_source.size(0)):
            for j in filter(lambda h: h>i, range(data_target.size(0))):

                W[i][j] = wasserstein_dist(data_source[i], data_target[j], metric, d).to(device)
                
    elif ind2 > ind1:
        
        for i in range(data_source.size(0)):
            for j in range(data_target.size(0)):
                
                W[i][j] = wasserstein_dist(data_source[i], data_target[j], metric, d).to(device)
            
    return W
            

In [26]:
def iterable(data, processes, metric, d):
    
    data = torch.stack(torch.chunk(data, processes, dim = 0), dim = 0)
    
    for i in range(data.size(0)):
        for j in range(data.size(0)):

            yield([data[i], data[j], metric, d, i, j, f'cuda:0'])

In [10]:
data, _ = torch.load('data/MNIST/test_eq_s=1500')
# data = data.to(device)
d = 2*28*28

In [32]:
import time
#Wasserstein distance 
# if __name__ == '__main__':
# size = data.size(0)

size = 100
metric = 'sqeuclidean'
W_dist = torch.zeros(size, size, dtype = torch.float)
processes = 5

ite = iterable(data[:size], processes, metric, d)

# mp.set_start_method('spawn')

st = time.time()
with mp.Pool(processes = processes) as p:

    W_dist = torch.cat(torch.chunk(torch.cat(p.starmap(fun, ite), dim = 1), processes, dim = 1), dim = 0)


et = time.time()
# print(W_dist)
print(et - st)

KeyboardInterrupt: 

In [15]:
print(W_dist)

tensor([[0.0000, 0.0706, 0.0827,  ..., 0.0519, 0.0946, 0.0658],
        [0.0000, 0.0000, 0.0653,  ..., 0.0510, 0.0663, 0.0477],
        [0.0000, 0.0000, 0.0000,  ..., 0.0827, 0.0460, 0.0748],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0891, 0.0498],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0737],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])
